In [4]:
#%matplotlib inline
#%matplotlib widget
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.widgets import Slider, Button, RadioButtons
import os,time,subprocess,glob
from PIL import Image

## set the following appropriately
PH_PROG = "CubicalRipser.exe"
#PH_PROG = "CR3.exe"
#PH_PROG = "./cubicalripser"
volfile = "vol.npy"
ph_out = "PH.npy"
volfile = "d:/ml/ipf/img.npy"
ph_out = "d:/ml/ipf/PH.npy"

In [ ]:
# load image files and stack into a 3D array
input_files = "*.jpg"
files = sorted([f for f in glob.glob(input_files)])
img = np.dstack([Image.open(f).convert('L') for f in files]).astype('f8')
np.save(volfile,img)
print(img.shape, img.dtype, np.min(img),np.max(img))

In [ ]:
# load image files and save in a 2D array
#input_file = "lena00.jpg"
#img = np.array(Image.open(input_file).convert('L'),dtype='f8')
#np.save(volfile,img)
#print(img.shape, img.dtype, np.min(img),np.max(img))

In [7]:
# compute PH (takes time)
start = time.time()
result = subprocess.run(PH_PROG+' --maxdim 0 --output '+ph_out+' '+volfile, shell=True)
print(result)
elapsed_time = time.time() - start
print ("elapsed_time:{} sec".format(elapsed_time))

CompletedProcess(args='CubicalRipser.exe --maxdim 0 --output d:/ml/ipf/PH.npy d:/ml/ipf/img.npy', returncode=0)
elapsed_time:56.03558421134949 sec


In [6]:
# load computed PH
pd = np.load(ph_out)
pd0 = pd[pd[:,0] == 0]
pd1 = pd[pd[:,0] == 1]
pd2 = pd[pd[:,0] == 2]
print("#0-cycle {}, #1-cycle {}, #2-cycle {}".format(len(pd0),len(pd1),len(pd2)))
# each line contains (dim,birth,death,x,y,z)
print(pd0[:5])
print(pd1[:5])
print(pd2[:5])

#0-cycle 499580, #1-cycle 759479, #2-cycle 170526
[[    0. -1286. -1266.   262.   164.    96.]
 [    0. -1285. -1263.   190.    78.   216.]
 [    0. -1268. -1263.   322.   128.   133.]
 [    0. -1292. -1263.   167.   298.   112.]
 [    0. -1265. -1260.   311.   156.   332.]]
[[1.000e+00 1.412e+03 1.420e+03 2.910e+02 5.400e+01 1.650e+02]
 [1.000e+00 1.170e+03 1.182e+03 1.750e+02 5.500e+01 2.400e+02]
 [1.000e+00 1.130e+03 1.153e+03 2.880e+02 5.200e+01 1.660e+02]
 [1.000e+00 1.125e+03 1.132e+03 2.900e+02 5.300e+01 1.630e+02]
 [1.000e+00 1.121e+03 1.226e+03 1.770e+02 5.400e+01 2.400e+02]]
[[   2. 1624. 1641.  271.  122.  216.]
 [   2. 1556. 1639.  293.   55.  165.]
 [   2. 1495. 1548.  295.   56.  163.]
 [   2. 1465. 1547.  252.   41.  242.]
 [   2. 1455. 1572.  206.   43.  271.]]


In [ ]:
# compute the heatmap of selected cycles
img = np.load(volfile)
mx,my,mz=img.shape
selected_cycle = np.zeros(img.shape)
h=3
min_life = 10
max_life =255
min_birth = 0
max_birth = 255
dimension = 0
pds = pd[pd[:,0] == dimension ]
pds = pds[min_life < pds[:,2]-pds[:,1]]
pds = pds[pds[:,2]-pds[:,1] < max_life]
pds = pds[min_birth < pds[:,1]]
pds = pds[pds[:,1] < max_birth]

print(pds.shape)
for c in pds:
    x,y,z=int(c[3]),int(c[4]),int(c[5])
#    selected_cycle[x,y,z] += 1
    selected_cycle[max(0,x-h):min(mx,x+h),max(0,y-h):min(my,y+h),max(0,z-h):min(mz,z+h)] += 1

print(np.min(selected_cycle),np.max(selected_cycle),np.sum(selected_cycle))

In [ ]:
# Visualise 
vol = selected_cycle
#vol = selected_cycle > 300
fig = plt.figure()
fig.subplots_adjust(left=0.25, bottom=0.25)
ax = plt.subplot(121)
ind = vol.shape[2]//2
l = ax.imshow(vol[:,:,ind])
ax = plt.subplot(122)
l2 = ax.imshow(img[:,:,ind])
def update(val):
    ind = int(slider.val)
    l.set_data(vol[:,:,ind])
    l2.set_data(img[:,:,ind])
    fig.canvas.draw()        
ax = fig.add_axes([0.25, 0.1, 0.65, 0.03])
slider = Slider(ax, 'index', 0, vol.shape[2] - 1, valinit=ind, valfmt='%i')
slider.on_changed(update)
plt.show()